<a href="https://colab.research.google.com/github/eyreynational/Final-Model-YOLOv8-CNN-ResNet-and-VGG16-/blob/main/VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, optimizers
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [ ]:
# Define paths
train_dir = '/content/drive/MyDrive/Master/Dataset/trian'  # Update with your path
test_dir = '/content/drive/MyDrive/Master/Dataset/test'


In [ ]:
# Data Augmentation for the Training Set
train_datagen = ImageDataGenerator(
    rescale=1./255,                # Normalization
    rotation_range=40,             # Random rotations
    width_shift_range=0.2,         # Horizontal shifts
    height_shift_range=0.2,        # Vertical shifts
    shear_range=0.2,               # Shear transformations
    zoom_range=0.2,                # Random zoom
    horizontal_flip=True,          # Flip horizontally
    fill_mode='nearest'            # Fill mode for shifts
)

In [ ]:
# For the Test Set, only rescale
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the training set
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'  # For multi-class classification
)

In [ ]:
# Load the test set
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
# Load the VGG16 model with pre-trained ImageNet weights, excluding the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))


In [ ]:
# Freeze the base model layers to prevent training on them
base_model.trainable = False

# Add custom top layers for your dataset
model = models.Sequential([
    base_model,  # Add the VGG16 base model
    layers.Flatten(),  # Flatten the output of VGG16
    layers.Dense(512, activation='relu'),  # Dense layer
    layers.Dropout(0.5),  # Dropout to prevent overfitting
    layers.Dense(5, activation='softmax')  # Output layer for 5 classes
])

In [ ]:
# Compile the model
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),  # Adjust the learning rate if necessary
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=70,  # Start with fewer epochs and increase if needed
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)


In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print(f"Test Accuracy: {test_acc}")

In [ ]:
# Predict on the test set
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
# Classification report for precision, recall, f1-score
print('Classification Report')
target_names = ['Powdery_mildew', 'Bacterial_spot', 'Late_blight', 'Healthy', 'Mosaic_virus']
print(classification_report(test_generator.classes, y_pred, target_names=target_names,digits=4))


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

# Get the confusion matrix
cm = confusion_matrix(test_generator.classes, y_pred)

# Class labels
class_names = ['Powdery_mildew', 'Bacterial_spot', 'Late_blight', 'Healthy', 'Mosaic_virus']

# Calculate per-class accuracy
print("Per-Class Accuracy:")
for i, class_name in enumerate(class_names):
    correct = cm[i, i]
    total = cm[i].sum()
    accuracy = correct / total if total > 0 else 0
    print(f"{class_name}: {accuracy:.4f}")


In [ ]:
# Confusion matrix for detailed error analysis
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))

In [ ]:
# Save the entire model to a file
model.save('https://drive.google.com/drive/folders/1tmOAHKL5YK_dxG9ufIbU_DDsIhSPcZZH/vgg16_model.h5')


In [ ]:
model.save('my_trained_model.h5')